# Solana Fee Market Bot

Introduction

This notebook demonstrates how to create a Discord bot that fetches data from a WebSocket and makes JSON-RPC requests, processing and displaying the data within Discord.  This would be useful for token traders and snipers who need the most up to date tip/fee markets while in active trading sessions.  Here's an step-by-step overview of this notebook:

	1.	Setup and Imports
	2.	Fetch Data from WebSocket
	3.	Get Priority Fee Estimate via JSON-RPC
	4.	Handle Discord Bot Events
	5.	Start the Bot

## 1. Setup and Imports
Make sure to replace `YOUR_BOT_TOKEN` and `YOUR_HELIUS_API_KEY` with your actual bot token and Helius API key.

In [ ]:
import discord
import asyncio
import websockets
import json
import requests
from datetime import datetime


TOKEN = 'YOUR_BOT_TOKEN'
WEBSOCKET_URL = 'ws://bundles-api-rest.jito.wtf/api/v1/bundles/tip_stream'
HELIUS_API_URL = 'https://mainnet.helius-rpc.com/?api-key=YOUR_HELIUS_API_KEY'

intents = discord.Intents.default()
client = discord.Client(intents=intents)

# Global list to store data
data_cache = []

## 2. Fetch Data from WebSocket
The following function connects to a WebSocket, sends a request, and retrieves the response.

In [ ]:
async def fetch_data():
    async with websockets.connect(WEBSOCKET_URL) as websocket:
        await websocket.send("Your request data if needed")
        response = await websocket.recv()
        data = json.loads(response)
        timestamp = datetime.utcnow()
        data_cache.append((timestamp, data))
        return data

## 3. Get Priority Fee Estimate via JSON-RPC
This function sends a JSON-RPC request to the Helius API and returns the priority fee estimate.

In [ ]:
def get_priority_fee_estimate():
    payload = {
        "jsonrpc": "2.0",
        "id": "1",
        "method": "getPriorityFeeEstimate",
        "params": [{
            "accountKeys": ["JUP6LkbZbjS1jKKwapdHNy74zcZ3tLUZoi5QNyVTaV4"],
            "options": {
                "includeAllPriorityFeeLevels": True
            }
        }]
    }
    
    headers = {
        'Content-Type': 'application/json'
    }
    
    response = requests.post(HELIUS_API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": "Failed to fetch priority fee estimate"}

## 4. Handle Discord Bot Events
The following event handlers manage messages and commands received by the bot.

In [ ]:
@client.event
async def on_ready():
    print(f'We have logged in as {client.user}')

@client.event
async def on_message(message):
    if message.author == client.user:
        return

    if message.content.startswith('!tips'):
        print(f"Received !tips command from {message.author}")
        data = await fetch_data()

        # Create an embed
        embed = discord.Embed(title="Jito Tip Percentiles", description="Here is the data fetched from Jito Labs", color=0x00ff00)
        
        for item in data:
            # Convert the time to a human-readable format
            time_value = item['time']
            dt = datetime.strptime(time_value, "%Y-%m-%dT%H:%M:%SZ")
            gmt_time_str = dt.strftime("%Y-%m-%d %H:%M:%S GMT")
            
            embed.add_field(name="Time", value=gmt_time_str, inline=False)
            embed.add_field(name="25th Percentile", value=f"{item['landed_tips_25th_percentile']:.6f}", inline=True)
            embed.add_field(name="50th Percentile", value=f"{item['landed_tips_50th_percentile']:.6f}", inline=True)
            embed.add_field(name="75th Percentile", value=f"{item['landed_tips_75th_percentile']:.6f}", inline=True)
            embed.add_field(name="95th Percentile", value=f"{item['landed_tips_95th_percentile']:.6f}", inline=True)
            embed.add_field(name="99th Percentile", value=f"{item['landed_tips_99th_percentile']:.6f}", inline=True)
            embed.add_field(name="EMA 50th Percentile", value=f"{item['ema_landed_tips_50th_percentile']:.6f}", inline=True)
        
        await message.channel.send(embed=embed)

    if message.content.startswith('!priorityfee'):
        print(f"Received !priorityfee command from {message.author}")
        fee_estimate = get_priority_fee_estimate()
        
        # Create an embed for the priority fee estimate
        embed = discord.Embed(title="Priority Fee Estimate", description="Here is the priority fee estimate fetched from Helius", color=0xff0000)
        
        if "error" in fee_estimate:
            embed.add_field(name="Error", value=fee_estimate["error"], inline=False)
        else:
            priority_fee_levels = fee_estimate.get("result", {}).get("priorityFeeLevels", {})
            embed.add_field(name="Min", value=f"{priority_fee_levels.get('min', 0.0) / 1_000_000_000:.9f}", inline=True)
            embed.add_field(name="Low", value=f"{priority_fee_levels.get('low', 0.0) / 1_000_000_000:.9f}", inline=True)
            embed.add_field(name="Medium", value=f"{priority_fee_levels.get('medium', 0.0) / 1_000_000_000:.9f}", inline=True)
            embed.add_field(name="High", value=f"{priority_fee_levels.get('high', 0.0) / 1_000_000_000:.9f}", inline=True)
            embed.add_field(name="Very High", value=f"{priority_fee_levels.get('veryHigh', 0.0) / 1_000_000_000:.9f}", inline=True)
            embed.add_field(name="Unsafe Max", value=f"{priority_fee_levels.get('unsafeMax', 0.0) / 1_000_000_000:.9f}", inline=True)
        
        await message.channel.send(embed=embed)

## 5. Start the Bot
Run the bot and handle asynchronous execution in Jupyter Notebook.

In [ ]:
# Start the bot
async def main():
    await client.start(TOKEN)

# Check if the script is running in a Jupyter notebook environment
if __name__ == "__main__" and "get_ipython" not in globals():
    # Normal script execution
    asyncio.run(main())
else:
    # Jupyter notebook or interactive environment
    loop = asyncio.get_event_loop()
    loop.create_task(main())